In [1]:
!pip install --quiet langchain langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00


In [11]:
import os

os.environ["OPENAI_API_TYPE"] = "azure_ad"
os.environ["OPENAI_API_VERSION"] = "2024-05-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = "This form the Azure portal - model deployment"
os.environ["AZURE_OPENAI_API_KEY"] = 'This form the Azure portal - model deployment'


In [12]:
from langchain_openai import AzureChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [13]:
llm = AzureChatOpenAI(
            temperature=1,
            azure_deployment="gpt-4o",
        )

## Chain 1: Synopsis Chain.

In [14]:
synopsis_prompt = PromptTemplate.from_template(
        """
            You are a playwright. Given the title of play, it is your job to write a synopsis for that title.

            Title: {title}

            Playwright: This is a synopsis for the above play:
        """
    )

In [15]:
synopsis_chain = LLMChain(llm=llm,
                          prompt=synopsis_prompt)

In [16]:
synopsis_chain

LLMChain(prompt=PromptTemplate(input_variables=['title'], template='\n            You are a playwright. Given the title of play, it is your job to write a synopsis for that title.\n        \n            Title: {title}\n            \n            Playwright: This is a synopsis for the above play:\n        '), llm=AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x780c242d0d60>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x780c240f0b80>, temperature=1.0, openai_api_key=SecretStr('**********'), openai_proxy='', azure_endpoint='https://mobility-sage-openai-services.openai.azure.com/', deployment_name='gpt-4o', openai_api_version='2024-05-01-preview', openai_api_type='azure_ad'))

## Chain 2: Review Chain.

In [17]:
review_prompt = PromptTemplate.from_template(
    """
        You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

        Play Synopsis:
        {synopsis}

        Review from a New York Times play critic of the above play:
    """
    )

In [18]:
review_chain = LLMChain(llm=llm,
                        prompt=review_prompt)


In [19]:
review_chain

LLMChain(prompt=PromptTemplate(input_variables=['synopsis'], template='\n        You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.\n    \n        Play Synopsis:\n        {synopsis}\n        \n        Review from a New York Times play critic of the above play:\n    '), llm=AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x780c242d0d60>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x780c240f0b80>, temperature=1.0, openai_api_key=SecretStr('**********'), openai_proxy='', azure_endpoint='https://mobility-sage-openai-services.openai.azure.com/', deployment_name='gpt-4o', openai_api_version='2024-05-01-preview', openai_api_type='azure_ad'))

# Simple Sequential Chain

In [21]:
from langchain.chains import SimpleSequentialChain

In [22]:
overall_chain = SimpleSequentialChain(
    chains = [synopsis_chain, review_chain],
    verbose = True
)

In [23]:
overall_chain.run("Tragedy at sunset on the beach")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SimpleSequentialChain chain...
**Tragedy at Sunset on the Beach**

**Playwright:**

*Synopsis:*

Set against the backdrop of a secluded coastal village, "Tragedy at Sunset on the Beach" unfolds over the course of a single, fateful summer evening. The play explores themes of love, loss, and the ripple effects of choices made in desperation.

The central characters are Emily, a young artist returning to her hometown after years in the city; Nathan, her estranged childhood love who's now a weary fisherman; and Laura, Nathan's vivacious but troubled wife, who harbors deep secrets of her own.

The play opens with Emily arriving at the beach, seeking solace and inspiration in the familiar sunsets that once inspired her art. In the fading light, she unexpectedly encounters Nathan, who is wrestling with the burdens of his failing marriage and his sense of duty to a life he no longer recognizes. Despite their unresolved past, there is an undeniable connection between them.

As 

'**A Sweeping Tale of Love and Loss: "Tragedy at Sunset on the Beach"**\n\nBy [Your Name]\n\nTaking cues from the timeless themes of classic drama, "Tragedy at Sunset on the Beach" by [Playwright\'s Name] delivers a reverberating punch to the gut, casting its audience into the churning sea of human emotion and fragility. The melancholic coastal village setting is not just a backdrop but a living, breathing character in itself, mirroring the tempestuous inner lives of the play’s trio of protagonists. Indeed, this play’s haunting echoes linger long after the final bow.\n\nThe narrative begins serenely enough, with Emily, a city-worn artist, returning to the village sunsets that once colored her canvases. From the moment she steps onto the sun-drenched sands, the stage is set for a convergence of past and present. Nathan, her childhood love turned weary fisherman, and Laura, his vibrant yet troubled spouse, form the rickety triangle at the heart of this emotional tempest.\n\nEmily’s encou